#  Capstone Project 2: Stock price forecasting
# Section 2: Data wrangling

In this notebook we focus on the data wrangling part of the project. We will collect data about historical stock prices and company earnings using Yahoo Finance and Alpha Vantage. The aim is to create a dataframe containing all the relevant information to train our models for stock price forecasting in future steps. We will focus on two stocks: Coca-Cola, a stock with a long history of solid growth, and Tesla, a newer and more volatile one. 

## 1. Coca-Cola stock

In [1]:


import os
import sys
import requests
import json
import numpy as np
import pandas as pd


### Data collection

We shall import the historical stock price data from Yahoo Finance. Let's start with the Coca-Cola stock, whose ticker is `KO`. Obviously the start date of the collected data will vary for the different stocks. Regarding the end date, we shall fix it as 31st December 2022.

In [2]:
import yfinance as yf

# Define the ticker symbol and end date
ticker = "KO"
end_date = "2022-12-31"

# Fetch the historical data using yfinance with start=None
data = yf.download(ticker, start=None, end=end_date)


[*********************100%***********************]  1 of 1 completed


In [3]:
# Make sure we have a dataframe type
type(data)

pandas.core.frame.DataFrame

Now we shall print out the head of our dataset `data`:

In [4]:
# Print out the head of the dataframe
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1962-01-02,0.263021,0.270182,0.263021,0.263021,0.048913,806400
1962-01-03,0.259115,0.259115,0.253255,0.257161,0.047823,1574400
1962-01-04,0.257813,0.261068,0.257813,0.259115,0.048187,844800
1962-01-05,0.259115,0.262370,0.252604,0.253255,0.047097,1420800
1962-01-08,0.251302,0.251302,0.245768,0.250651,0.046613,2035200


Let's check for missing values and NaN values:

In [5]:
# Check for missing values in the DataFrame
missing_values = data.isnull().sum()

# Print the missing values count
print(missing_values)

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


In [6]:
# Check for NaN values in the DataFrame
nan_values = data.isna().sum()

# Print the NaN values count
print(nan_values)

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


We have no missing or NaN values in this original dataset for stock prices. However, in future steps we will find NaN values for certain rows when computing financial indicators, and these will need to be dropped.

With this information about stock prices we can compute financial indicators, which will be used as features. However, we would like 
to have additional features that are not directly related to the stock price, such as the earnings per share (EPS), which enables us to compute the P/E ratio --a commonly used metric to value a publicly traded company. Here we shall 
import this information from the Alpha Vantage API.

In order to avoid posting publicly our API key, we shall store it in a `.env` file and use the `python-dotenv` package to load the key into environment variables.

In [7]:
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv('API_KEY')

In [8]:
# Obtaining earnings per share (EPS)

BASE_URL = "https://www.alphavantage.co/query?"

def earnings_history_api(api_key, symbol):
    assert symbol is not None
    symbol = symbol.strip().upper()

    url = f"{BASE_URL}function=EARNINGS&symbol={symbol}&apikey={api_key}"

    response = requests.get(url)
    data = response.json()

    if 'quarterlyEarnings' in data:
        earnings_data = data['quarterlyEarnings']
        df = pd.DataFrame(earnings_data)
        return df
    else:
        print("No earnings data found for the specified symbol.")
        return None

df_earnings = earnings_history_api(API_KEY, "KO")

# Print dataframe with EPS
df_earnings


,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,2023-03-31,2023-04-24,0.68,0.65,0.03,4.6154
1,2022-12-31,2023-02-14,0.45,0.45,0,0
2,2022-09-30,2022-10-25,0.69,0.64,0.05,7.8125
3,2022-06-30,2022-07-26,0.7,0.67,0.03,4.4776
4,2022-03-31,2022-04-25,0.64,0.58,0.06,10.3448
...,...,...,...,...,...,...
104,1997-03-31,1997-04-14,0.2,0.2,0,0
105,1996-12-31,1997-01-31,0.16,0.16,0,0
106,1996-09-30,1996-10-15,0.2,0.19,0.01,5.2632
107,1996-06-30,1996-07-15,0.21,0.21,0,0


We see that the first row in this dataframe dates from 1996-03-01, which is way more recent that the first data within the historical stock prices dataframe. Still, the EPS dataframe provides us with 26 years of data, which in combination with the stock prices data restricted to this period should be sufficient to effectively train our models. 

In the dataframes of Alpha Vantage missing values appear as "None". We have to check that there are no None values in `df_earnings`:

In [9]:
(df_earnings == 'None').sum().sum()

0

###  Data organization and data cleaning

We now would like to incorporate the `reportedEPS` column in our dataframe for stock prices `data`, in such a way that every row is assigned the reported EPS that corresponds to the row's date. In order to do this, first we note that the data type of the `reportedDate` column is different from the data type of the index in our main dataframe: the former is an Object type whereas the latter is a datetime64 type.

In [10]:
print(df_earnings['reportedDate'].dtype)

object


In [11]:
print(data.index.dtype)

datetime64[ns]


So first we need to convert the data type of the `reportedDate` column to a datetime64 type. 

In [12]:
# Convert the reportedDate column to a datetime64 type
df_earnings['reportedDate'] = pd.to_datetime(df_earnings['reportedDate']) 

# Check data type
print(df_earnings['reportedDate'].dtype)


datetime64[ns]


Then we promote the `reportedDate` column to being the index:

In [13]:
# Set the `reportedDate' column as the index
df_earnings = df_earnings.set_index('reportedDate')

# Print the dataframe and check
df_earnings.head()

,fiscalDateEnding,reportedEPS,estimatedEPS,surprise,surprisePercentage
reportedDate,,,,,
2023-04-24,2023-03-31,0.68,0.65,0.03,4.6154
2023-02-14,2022-12-31,0.45,0.45,0,0
2022-10-25,2022-09-30,0.69,0.64,0.05,7.8125
2022-07-26,2022-06-30,0.7,0.67,0.03,4.4776
2022-04-25,2022-03-31,0.64,0.58,0.06,10.3448


Let's sort the dataframe in ascending order:

In [14]:
df_earnings.sort_index(ascending=True, inplace=True)

In [15]:
df_earnings.head()

,fiscalDateEnding,reportedEPS,estimatedEPS,surprise,surprisePercentage
reportedDate,,,,,
1996-04-17,1996-03-31,0.14,0.14,0,0
1996-07-15,1996-06-30,0.21,0.21,0,0
1996-10-15,1996-09-30,0.2,0.19,0.01,5.2632
1997-01-31,1996-12-31,0.16,0.16,0,0
1997-04-14,1997-03-31,0.2,0.2,0,0


We are only interested in adding the `reportedEPS` column, so we shall drop the others.

In [16]:
df_earnings.drop(["fiscalDateEnding", "estimatedEPS", "surprise", "surprisePercentage"], axis=1, inplace=True)

Let's also change the name of `reportedEPS` column to `Reported EPS`:

In [17]:
df_earnings.rename(columns={'reportedEPS': 'Reported EPS'}, inplace=True)


Finally we merge the two dataframes on their index and forward fill the missing values in the `Reported EPS` column

In [18]:
# Merge 'data' and 'df_earnings' based on their index
merged_df = data.join(df_earnings, how="left")

# Forward fill the missing values in the "Reported EPS" column
merged_df["Reported EPS"].fillna(method="ffill", inplace=True)

Let's have a look at the head of our merged dataframe:

In [19]:
merged_df.head()

,Open,High,Low,Close,Adj Close,Volume,Reported EPS
Date,,,,,,,
1962-01-02,0.263021,0.270182,0.263021,0.263021,0.048913,806400,NaN
1962-01-03,0.259115,0.259115,0.253255,0.257161,0.047823,1574400,NaN
1962-01-04,0.257813,0.261068,0.257813,0.259115,0.048187,844800,NaN
1962-01-05,0.259115,0.262370,0.252604,0.253255,0.047097,1420800,NaN
1962-01-08,0.251302,0.251302,0.245768,0.250651,0.046613,2035200,NaN


As we mentioned earlier, the dataframe containing information about EPS only has entries from 1996-04-07, so we will drop the rows up to that point. This will yield a dataframe with sufficent information to fit our models reasonably well.

In [20]:
# Drop rows with NaN values
df = merged_df.dropna()

# Print the final form of our dataframe
df

,Open,High,Low,Close,Adj Close,Volume,Reported EPS
Date,,,,,,,
1996-04-17,20.281250,20.281250,19.843750,20.031250,10.141943,8906000,0.14
1996-04-18,20.031250,20.156250,19.843750,19.875000,10.062835,9608000,0.14
1996-04-19,19.875000,20.062500,19.562500,19.687500,9.967903,13010400,0.14
1996-04-22,19.875000,20.187500,19.875000,20.156250,10.205235,7160800,0.14
1996-04-23,20.156250,20.281250,20.062500,20.250000,10.252699,6218800,0.14
...,...,...,...,...,...,...,...
2022-12-23,63.500000,63.869999,63.200001,63.820000,62.855492,6463300,0.69
2022-12-27,63.930000,64.290001,63.709999,64.209999,63.239597,7320700,0.69
2022-12-28,64.459999,64.650002,63.490002,63.570000,62.609272,7159400,0.69


Let's now check that the `Reported EPS` column has been filled correctly when the values are updated. For instance, let's print the transition from 2022-07-25 to 2022-07-26:

In [21]:
# Print our final dataframe on the selected transition date
df.tail(112)

,Open,High,Low,Close,Adj Close,Volume,Reported EPS
Date,,,,,,,
2022-07-25,61.549999,62.299999,61.310001,62.189999,60.378578,14941700,0.64
2022-07-26,62.750000,63.799999,62.529999,63.209999,61.368870,20118000,0.7
2022-07-27,62.880001,63.189999,61.790001,63.009998,61.174698,12142900,0.7
2022-07-28,62.919998,64.250000,62.889999,64.059998,62.194111,10837000,0.7
2022-07-29,63.709999,64.290001,63.630001,64.169998,62.300907,13734200,0.7
...,...,...,...,...,...,...,...
2022-12-23,63.500000,63.869999,63.200001,63.820000,62.855492,6463300,0.69
2022-12-27,63.930000,64.290001,63.709999,64.209999,63.239597,7320700,0.69
2022-12-28,64.459999,64.650002,63.490002,63.570000,62.609272,7159400,0.69


We see that the `Reported EPS` column is correctly updated on the correspoding date accoridng to the information in the dataframe `df_earnings`. 

### 2. Tesla stock

We shall repeat the same steps we took in the previous section but focusing this time on the TSLA ticker.

### Data collection

In [22]:
data2 = yf.download('TSLA', start=None, end=end_date)
data2

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-06-29,1.266667,1.666667,1.169333,1.592667,1.592667,281494500
2010-06-30,1.719333,2.028000,1.553333,1.588667,1.588667,257806500
2010-07-01,1.666667,1.728000,1.351333,1.464000,1.464000,123282000
2010-07-02,1.533333,1.540000,1.247333,1.280000,1.280000,77097000
2010-07-06,1.333333,1.333333,1.055333,1.074000,1.074000,103003500
...,...,...,...,...,...,...
2022-12-23,126.370003,128.619995,121.019997,123.150002,123.150002,166989700
2022-12-27,117.500000,119.669998,108.760002,109.099998,109.099998,208643400
2022-12-28,110.349998,116.269997,108.239998,112.709999,112.709999,221070500


In [23]:
# Check for missing values in the DataFrame
missing_values = data2.isnull().sum()

# Print the missing values count
print(missing_values)

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


In [24]:
# Check for NaN values in the DataFrame
nan_values = data2.isna().sum()

# Print the NaN values count
print(nan_values)

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


In [25]:
df_earnings2 = earnings_history_api(API_KEY, "TSLA")
df_earnings2

,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,2023-03-31,2023-04-19,0.85,0.85,0,0
1,2022-12-31,2023-01-25,1.19,1.13,0.06,5.3097
2,2022-09-30,2022-10-19,1.05,0.99,0.06,6.0606
3,2022-06-30,2022-07-20,0.76,0.6,0.16,26.6667
4,2022-03-31,2022-04-20,1.07,0.75,0.32,42.6667
5,2021-12-31,2022-01-26,0.85,0.79,0.06,7.5949
6,2021-09-30,2021-10-20,0.62,0.53,0.09,16.9811
7,2021-06-30,2021-07-26,0.48,0.33,0.15,45.4545
8,2021-03-31,2021-04-26,0.31,0.26,0.05,19.2308
9,2020-12-31,2021-01-27,0.27,0.34,-0.07,-20.5882


Now we check for None values in `df_earnings2`. We see that there are 3 entries with a `None` value:

In [26]:
# Check for missing values in 'df_earnings2'
(df_earnings2 == 'None').sum().sum()


3

However, these are the first entries for `estimatedEPS`, `surprise` and `surprisePErcentage`, which obviously must be undefined as there is no possibility of prior estimation. In any case, these `None` values do not affect our analysis, as we are just picking the `reportedEPS` column.

### Data organization and cleaning

Let's incorporate the `reportedEPS` column into the dataframe `data2` containing stock price data. We follow the same steps as in the previous case.

In [27]:
# Convert the reportedDate column to a datetime64 type
df_earnings2['reportedDate'] = pd.to_datetime(df_earnings2['reportedDate']) 

# Check data type
print(df_earnings2['reportedDate'].dtype)


datetime64[ns]


In [28]:
# Set the `reportedDate' column as the index
df_earnings2 = df_earnings2.set_index('reportedDate')

# Sort in ascending order
df_earnings2.sort_index(ascending=True, inplace=True)

# Print the dataframe and check
df_earnings2.head()

,fiscalDateEnding,reportedEPS,estimatedEPS,surprise,surprisePercentage
reportedDate,,,,,
2010-08-05,2010-06-30,-0.0271,None,None,None
2010-11-09,2010-09-30,-0.02,-0.03,0.01,33.3333
2011-02-15,2010-12-31,-0.03,-0.03,0,0
2011-05-04,2011-03-31,-0.03,-0.03,0,0
2011-08-03,2011-06-30,-0.04,-0.03,-0.01,-33.3333


In [29]:
# Drop columns different from "reportedEPS" and rename to "Reported EPS"
df_earnings2.drop(["fiscalDateEnding", "estimatedEPS", "surprise", "surprisePercentage"], axis=1, inplace=True)
df_earnings2.rename(columns={'reportedEPS': 'Reported EPS'}, inplace=True)


In [30]:
# Merge 'data' and 'df_earnings' based on their index
merged_df2 = data2.join(df_earnings2, how="left")

# Forward fill the missing values in the "Reported EPS" column
merged_df2["Reported EPS"].fillna(method="ffill", inplace=True)

In [31]:
merged_df2.head()

,Open,High,Low,Close,Adj Close,Volume,Reported EPS
Date,,,,,,,
2010-06-29,1.266667,1.666667,1.169333,1.592667,1.592667,281494500,NaN
2010-06-30,1.719333,2.028000,1.553333,1.588667,1.588667,257806500,NaN
2010-07-01,1.666667,1.728000,1.351333,1.464000,1.464000,123282000,NaN
2010-07-02,1.533333,1.540000,1.247333,1.280000,1.280000,77097000,NaN
2010-07-06,1.333333,1.333333,1.055333,1.074000,1.074000,103003500,NaN


We observe that the first entries of `merged_df2` have NaN values for `Reported EPS`, as the first reported EPS happened after the stock became publicly traded. So we will drop these rows.

In [32]:
# Drop rows with NaN values
df2 = merged_df2.dropna()

# Print the final form of our dataframe
df2

,Open,High,Low,Close,Adj Close,Volume,Reported EPS
Date,,,,,,,
2010-08-05,1.436000,1.436667,1.336667,1.363333,1.363333,11943000,-0.0271
2010-08-06,1.340000,1.344000,1.301333,1.306000,1.306000,11128500,-0.0271
2010-08-09,1.326667,1.332000,1.296667,1.306667,1.306667,12190500,-0.0271
2010-08-10,1.310000,1.310000,1.254667,1.268667,1.268667,19219500,-0.0271
2010-08-11,1.246000,1.258667,1.190000,1.193333,1.193333,11964000,-0.0271
...,...,...,...,...,...,...,...
2022-12-23,126.370003,128.619995,121.019997,123.150002,123.150002,166989700,1.05
2022-12-27,117.500000,119.669998,108.760002,109.099998,109.099998,208643400,1.05
2022-12-28,110.349998,116.269997,108.239998,112.709999,112.709999,221070500,1.05
